In [31]:
''' Imports '''

import pandas as pd
import polars as pl
import numpy as np
import math
from datetime import time

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats.mstats import trimmed_var
from scipy.stats import percentileofscore

from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, SpectralClustering, DBSCAN, HDBSCAN, OPTICS, estimate_bandwidth
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler

from prep_data import load_pbp_participation_data, load_stats_team_tendencies_offense, load_stats_team_tendencies_defense

In [2]:
offense_tendencies = load_stats_team_tendencies_offense()

print(offense_tendencies.head().to_string())

                Games  Drives  Plays  Neutral_Down_Plays  Pass_Plays  Neutral_Down_Pass  Pass_Attempts  QBScrambles     IAY  IAY_ToSticks  TotalTimeToThrow  Pass_BehindLOS  Pass_Deep  Sacks  Rush_Plays  Rush_Attempts  Rush_Inside  Rush_Outside  Plays_11_Personnel  Plays_Heavy_Personnel  Plays_Mult_RBs  Plays_Zero_RBs  Plays_Mult_TEs  Plays_Zero_TEs  Plays_Extra_OL  Plays / Game  Drives / Game    % Pass  % Pass Neutral Downs  Scrambles / Game      ADOT  ADOT to Sticks  Avg Time to Throw  % Passes Behind LOS  % Passes Deep  % Rush Inside  % Rush Outside  % Plays 11 Personnel  % Plays Heavy Personnel  % Plays Mult RBs  % Plays Zero RBs  % Plays Mult TEs  % Plays Zero TEs  % Plays Extra OL  Shotgun Plays  Under Center Plays  Shotgun Neutral_Down_Plays  Under Center Neutral_Down_Plays  Shotgun % Pass  Under Center % Pass  % Under Center  % Shotgun  % Under Center Neutral Downs  % Shotgun Neutral Downs  MaxTargets  MaxTargetShare  N_Receivers_FivePctTargetShare  MaxRushAttempts  MaxRushAttem

In [9]:
''' Features '''
# '% Pass Neutral Downs', '% Under Center Neutral Downs', '% Shotgun Neutral Downs',

OFFENSE_FEATURES = [
    'Plays / Game', 'Drives / Game', 
    '% Pass',  'Scrambles / Game',
    '% Plays 11 Personnel', '% Plays Heavy Personnel', '% Plays Mult RBs', '% Plays Zero RBs', '% Plays Mult TEs', '% Plays Zero TEs', '% Plays Extra OL',
    '% Under Center', '% Shotgun', 'Shotgun % Pass', 'Under Center % Pass',
    'ADOT', 'ADOT to Sticks', 'Avg Time to Throw', '% Passes Behind LOS', '% Passes Deep', 'MaxTargetShare',
    '% Rush Inside', '% Rush Outside', 'MaxRushAttemptsShare',
]

VIZ_FEATURES = ['Plays / Game', '% Pass', 'Scrambles / Game', 
                '% Plays Plays_11_Personnel',
                '% Under Center', 'ADOT', 'Avg Time to Throw', 'MaxTargetShare', 
                '% Rush Outside', 'MaxRushAttemptsShare']

# VIZ_FEATURES = ['Plays / Game', '% Pass', 'Scrambles / Game', 
#                 '% Plays Plays_11_Personnel', '% Plays Plays_Mult_RBs', '% Plays Plays_Mult_TEs',
#                 '% Under Center', 'Shotgun % Pass', 'Under Center % Pass', 
#                 'ADOT', 'Avg Time to Throw', '% Passes Behind LOS', '% Passes Deep', 'MaxTargetShare', 
#                 '% Rush Outside', 'MaxRushAttemptsShare']

# Visualize

In [4]:
''' Correlation Matrix '''

corr_matrix = offense_tendencies[OFFENSE_FEATURES].corr()

fig = px.imshow(
    corr_matrix,
    color_continuous_scale=px.colors.diverging.PRGn,
    aspect="auto"
)
fig.update_xaxes(side="top")
fig.update_coloraxes(
    cmid=0,
    showscale=False,
)
fig.update_layout(
    title='Feature Correlations',
    margin=dict(r=25, b=25)
)
fig.show()

In [5]:
''' Variance '''

# Calculate variance, get 10 largest features
top_ten_variance = offense_tendencies[OFFENSE_FEATURES].var().sort_values().tail(10)

# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_variance,
    y=top_ten_variance.index,
    title="Offense Tendencies: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Features")
fig.show()

# Calculate trimmed variance
top_ten_trim_variance = offense_tendencies[OFFENSE_FEATURES].apply(trimmed_var).sort_values().tail(10)

# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_variance,
    y=top_ten_trim_variance.index,
    title="Offense Tendencies: High Variance Features (Trimmed)"
)
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Features")
fig.show()


In [7]:
fig = px.histogram(
    data_frame=offense_tendencies,
    x='Plays / Game',
    title='Plays / Game'
)
fig.show()

In [ ]:
''' t-SNE '''



# Model Preprocessing

In [10]:
''' Transform and Scale '''

# ## Log transform data
# transformed_data = pd.DataFrame(np.log(offense_tendencies[OFFENSE_FEATURES]), columns=OFFENSE_FEATURES).replace(math.inf, 0).replace(-(math.inf), 0)

## Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(offense_tendencies[OFFENSE_FEATURES])

# Put `scaled_data` into DataFrame
scaled_data_df = pd.DataFrame(scaled_data, columns=OFFENSE_FEATURES)

print("scaled DF type:", type(scaled_data_df))
print("scaled DF shape:", scaled_data_df.shape)
scaled_data_df.head()

scaled DF type: <class 'pandas.core.frame.DataFrame'>
scaled DF shape: (224, 24)


,Plays / Game,Drives / Game,% Pass,Scrambles / Game,% Plays 11 Personnel,% Plays Heavy Personnel,% Plays Mult RBs,% Plays Zero RBs,% Plays Mult TEs,% Plays Zero TEs,...,Under Center % Pass,ADOT,ADOT to Sticks,Avg Time to Throw,% Passes Behind LOS,% Passes Deep,MaxTargetShare,% Rush Inside,% Rush Outside,MaxRushAttemptsShare
0,-2.331593,2.343784,-0.039531,-0.796092,0.375049,-0.092093,0.152784,-0.527200,-0.170789,-0.524651,...,0.367672,0.523527,0.517104,-0.171047,0.259090,0.241612,0.554760,-2.019394,-2.054735,1.714133
1,0.919371,1.291758,1.210869,0.118792,-1.451306,-0.379128,-0.396390,0.396326,-0.208703,7.602769,...,-0.266346,-0.691822,-0.665958,-0.475782,1.573069,0.271512,-0.736058,-2.837903,1.963484,-1.725784
2,1.569563,0.502738,0.770569,1.948560,-0.594612,-0.141177,-0.454407,-0.421931,0.158448,3.185271,...,1.319149,-0.015061,-0.386421,-0.010418,0.587780,-0.088111,1.282674,-2.332227,0.749015,0.197049
3,0.507350,-0.518347,0.777948,0.483550,-0.436346,-0.232420,-0.346630,-0.057475,-0.011029,3.191052,...,-0.346983,-0.306309,-0.245658,-0.553849,1.821468,1.365970,-1.358682,-2.355040,0.313172,-0.613659
4,1.147112,0.843099,0.854663,0.387875,0.197556,-0.184903,-0.436722,-0.527200,0.072687,0.924360,...,-2.479953,-0.970470,-1.079155,-1.117736,1.134455,0.196839,-1.345980,-1.439858,-0.504901,-0.343834


In [7]:
''' Variance of Scaled / Transformed Data '''

# Calculate variance, get 10 largest features
top_ten_variance = scaled_data_df.var().sort_values().tail(10)

# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_variance,
    y=top_ten_variance.index,
    title="Scaled Data: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Features")
fig.show()

# Feature Selection

1. Raw Features
    1. Could do Pearson feature selection variant; instead of features with highest correlation to Y, features with most variance, then remove features with collinearity > some threshold
1. PCA

In [11]:
''' PCA '''

# Instantiate transformer
pca = PCA(random_state=42)

# Transform data with pa
pca_component_data = pca.fit_transform(scaled_data_df)

print('Total variance:', scaled_data_df.var().sum())
print(f'Singular values:\n', pca.singular_values_)
print(f'Explained variance:\n', pca.explained_variance_.round(5))
print(f'Ratio:\n', pca.explained_variance_ratio_.round(3))
print(pca.feature_names_in_)

# Create horizontal bar chart of explained variance
fig = px.line(
    x=[i + 1 for i in range(len(pca.explained_variance_ratio_))],
    y=pca.explained_variance_ratio_.cumsum(),
    title="Explained variance"
)
fig.update_layout(xaxis_title="Principal Component", yaxis_title="Cumulative Explained Variance (%)")
fig.show()

Total variance: 24.10762331838565
Singular values:
 [2.78336619e+01 2.68058057e+01 2.50252777e+01 2.02933055e+01
 1.94727558e+01 1.82699157e+01 1.70402455e+01 1.63405143e+01
 1.59044997e+01 1.44548814e+01 1.36975454e+01 1.34397006e+01
 1.25450995e+01 1.14254609e+01 1.11387505e+01 1.07372387e+01
 9.60174666e+00 8.57664446e+00 5.46656752e+00 3.34913831e+00
 2.42907134e+00 1.74699247e+00 1.03067296e+00 2.85698686e-15]
Explained variance:
 [3.47405 3.2222  2.80836 1.84672 1.7004  1.49682 1.30211 1.19737 1.13432
 0.93697 0.84136 0.80998 0.70574 0.58539 0.55638 0.51699 0.41342 0.32986
 0.13401 0.0503  0.02646 0.01369 0.00476 0.     ]
Ratio:
 [0.144 0.134 0.116 0.077 0.071 0.062 0.054 0.05  0.047 0.039 0.035 0.034
 0.029 0.024 0.023 0.021 0.017 0.014 0.006 0.002 0.001 0.001 0.    0.   ]
['Plays / Game' 'Drives / Game' '% Pass' 'Scrambles / Game'
 '% Plays 11 Personnel' '% Plays Heavy Personnel' '% Plays Mult RBs'
 '% Plays Zero RBs' '% Plays Mult TEs' '% Plays Zero TEs'
 '% Plays Extra OL' '%

In [38]:
''' PCA - final '''

# Set number of PCA components to use after initial try
PCA_N_COMPONENTS = 8
PCA_COMPONENT_COLS = [f'PCA Component {n}' for n in range(1, PCA_N_COMPONENTS + 1)]

# Instantiate transformer
pca_final = PCA(n_components=PCA_N_COMPONENTS, random_state=42)

# Transform sku profiles
pca_component_data_final = pca_final.fit_transform(scaled_data_df)

# Evaluate components
total_variance = scaled_data_df.var().sum()
expl_variance = pca_final.explained_variance_.sum()

print(f'Data set variance: {total_variance:,.3f}')
print(f'PCA explained variance: {expl_variance:,.3f} ({round((expl_variance / total_variance) * 100, 2)}%)')

pcs = pd.DataFrame(pca_final.components_, columns=OFFENSE_FEATURES)

# Create bar charts of contribution
for n in range(2):      #PCA_N_COMPONENTS):
    pc = pcs.transpose()[n].sort_values(ascending=False)

    fig = px.bar(
        x=pc,
        y=pc.index,
        title=f"PC{n+1}: Greatest contributors"
    )
    fig.update_layout(
        xaxis_title="Correlation", 
        yaxis_title="Features", 
        yaxis={'dtick': 1, 'categoryorder':'total ascending'},
    )
    fig.show()

    comp_expl_variance = pca_final.explained_variance_[n]
    print(f'Explained variance: {comp_expl_variance:,} ({round((comp_expl_variance / total_variance) * 100, 2)}%)')
                                                                                                                                                                                                              
# Make df of PCA scores
pca_component_df = pd.DataFrame(data=pca_component_data_final, columns=PCA_COMPONENT_COLS)
print(pca_component_df.shape)
print(pca_component_df.head().to_string())

# Add PCA scores to original dataframe
offense_tendencies = offense_tendencies.drop(columns=list(filter(lambda x: x.startswith("Component"), offense_tendencies.columns)))
offense_tendencies = offense_tendencies.reset_index().merge(pca_component_df, left_index=True, right_index=True, how='left').set_index(['posteam', 'season'])


print(f'PCA values')
print(offense_tendencies.head().to_string())

Data set variance: 24.108
PCA explained variance: 17.048 (70.72%)


Explained variance: 3.4740481340483558 (14.41%)


Explained variance: 3.222202788519839 (13.37%)
(224, 8)
   PCA Component 1  PCA Component 2  PCA Component 3  PCA Component 4  PCA Component 5  PCA Component 6  PCA Component 7  PCA Component 8
0        -1.174916        -1.414879         1.749422        -0.534725         0.874502        -0.466387        -1.961966        -0.768966
1         2.907426         1.041547        -2.534225         1.033480        -1.286341         6.208372        -1.656764         3.034321
2         2.909212         1.744247        -1.583371         1.825540        -0.306463         2.587239         0.286811         2.006733
3         2.968790         1.307310        -2.042675         0.444853        -0.356500         2.745625        -1.558250         1.592760
4         3.112452        -0.114299        -2.719902         0.835783        -1.142196         0.693303        -1.657998        -0.695891
PCA values
                Games  Drives  Plays  Neutral_Down_Plays  Pass_Plays  Neutral_Down_Pass  Pass_Attempts  Q

/var/folders/68/537rmjg936dfpk9xqgf5vks40000gn/T/ipykernel_37800/3672584314.py:48: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'PCA Component 2_x', 'PCA Component 3_x', 'PCA Component 1_x'} in the result is deprecated and will raise a MergeError in a future version.



In [39]:
''' t-SNE for Visualization '''

PERPLEXITY = 30
TSNE_N_COMPONENTS = 2
TSNE_N_COMPONENT_NAMES = [f'TSNE Component {n+1}' for n in range(TSNE_N_COMPONENTS)]

# Model
tsne_model = TSNE(n_components=TSNE_N_COMPONENTS, perplexity=PERPLEXITY, random_state=42)

# Fit
y = tsne_model.fit_transform(scaled_data_df)
print(f'Divergence:', tsne_model.kl_divergence_)

# Results df
tsne_df = pd.DataFrame(y, columns=TSNE_N_COMPONENT_NAMES)

print(tsne_df.shape)
print(tsne_df.head().to_string())

Divergence: 0.9622150659561157
(224, 2)
   TSNE Component 1  TSNE Component 2
0          6.305297         -1.726348
1        -11.902147         -1.672906
2        -11.353816         -1.134172
3        -11.203395         -1.481916
4         -9.270196         -2.297225


In [28]:
''' KMeans - Raw Features vs. PCA '''

input_dfs = [scaled_data_df, pca_component_df]
titles = ['Raw Features', 'PCA']

for i in range(len(input_dfs)):

    kmeans_input = input_dfs[i].copy()
    title = titles[i]
    
    ## Experiment ##
    print(f'------------ {title} ------------')

    # Try kmeans clustering with up to 20 clusters, keep track of inertia (basically cluster variance)
    n_clusters = list(range(3,10))
    inertia_values = []
    silhouette_scores = []
    best_ss = 0
    best_n_clusters = 0

    for n in n_clusters:
        # Model
        kmeans = KMeans(n_clusters=n, n_init='auto', init='k-means++', random_state=42)

        # Fit
        kmeans.fit(kmeans_input)

        # Score
        ss = silhouette_score(kmeans_input, kmeans.labels_)   #, sample_size=int(len(pca_component_df) * 0.25))

        if ss > best_ss:
            best_ss = ss
            best_n_clusters = n

        inertia_values.append(kmeans.inertia_)
        silhouette_scores.append(ss)

    # Create scatter of inertia
    fig = px.line(
        x=n_clusters,
        y=inertia_values,
        title="Kmeans - Inertia by Number of Clusters"
    )
    fig.update_layout(xaxis_title="Num Clusters", yaxis_title="Inertia")
    fig.show()

    # Create a line plot of `silhouette_scores` vs `n_clusters`
    fig = px.line(
        x=n_clusters,
        y=silhouette_scores,
        title="K-Means Model: Silhouette Score vs Number of Clusters"
    )
    fig.update_layout(xaxis_title="Num Clusters", yaxis_title="Silhouette Score")
    fig.show()

    ## Final Clustering ##

    N_CLUSTERS = best_n_clusters
    print(f'N_CLUSTERS: {N_CLUSTERS}')

    # Once optimal num clusters is found, create the final cluster model
    kmeans_final = KMeans(n_clusters=N_CLUSTERS, n_init='auto', init='k-means++', random_state=42)
    kmeans_final.fit(kmeans_input)

    # Find distances to centroids
    labels = kmeans_final.labels_
    distances_array = kmeans_final.transform(kmeans_input)

    kmeans_input['Cluster KMEANS'] = labels + 1
    kmeans_input['Cluster KMEANS'] = kmeans_input['Cluster KMEANS'].astype(str)

    ## t-SNE for Visualization ##
    tsne_n_components = 3

    # Model
    tsne_model = TSNE(n_components=tsne_n_components, perplexity=30, random_state=42)

    # Fit
    y = tsne_model.fit_transform(kmeans_input)
    print(f'Divergence:', tsne_model.kl_divergence_)

    # Add components to input DF
    kmeans_input = kmeans_input.drop(columns=list(filter(lambda x: x.startswith("TSNE"), kmeans_input.columns)))
    tsne_df = pd.DataFrame(y, columns=[f'TSNE Component {n}' for n in range(1,tsne_n_components+1)])
    kmeans_input = pd.concat([kmeans_input, tsne_df], axis=1)
    
    print(kmeans_input.head().to_string())

    ## Visualize ##
    fig = go.Figure()
    if tsne_n_components == 2:
        fig = px.scatter(
            data_frame=kmeans_input,
            x='TSNE Component 1', #kmeans_input.columns[0],
            y='TSNE Component 2', #kmeans_input.columns[1],
            color='Cluster KMEANS',
        )
    else:
        fig = px.scatter_3d(
            data_frame=kmeans_input,
            x='TSNE Component 1', #kmeans_input.columns[0],
            y='TSNE Component 2', #kmeans_input.columns[1],
            z='TSNE Component 3', #kmeans_input.columns[2],
            color='Cluster KMEANS',
        )
    fig.show()

------------ Raw Features ------------


N_CLUSTERS: 6
Divergence: 0.8771711587905884
   Plays / Game  Drives / Game    % Pass  Scrambles / Game  % Plays 11 Personnel  % Plays Heavy Personnel  % Plays Mult RBs  % Plays Zero RBs  % Plays Mult TEs  % Plays Zero TEs  % Plays Extra OL  % Under Center  % Shotgun  Shotgun % Pass  Under Center % Pass      ADOT  ADOT to Sticks  Avg Time to Throw  % Passes Behind LOS  % Passes Deep  MaxTargetShare  % Rush Inside  % Rush Outside  MaxRushAttemptsShare Cluster KMEANS  TSNE Component 1  TSNE Component 2  TSNE Component 3
0     -2.331593       2.343784 -0.039531         -0.796092              0.375049                -0.092093          0.152784         -0.527200         -0.170789         -0.524651         -0.664734        1.072777  -1.072777        1.035049             0.367672  0.523527        0.517104          -0.171047             0.259090       0.241612        0.554760      -2.019394       -2.054735              1.714133              5         16.129128        -26.990572         61.1974

------------ PCA ------------


N_CLUSTERS: 3
Divergence: 0.7002525925636292
   PCA Component 1  PCA Component 2  PCA Component 3  PCA Component 4  PCA Component 5  PCA Component 6  PCA Component 7  PCA Component 8 Cluster KMEANS  TSNE Component 1  TSNE Component 2  TSNE Component 3
0        -1.174916        -1.414879         1.749422        -0.534725         0.874502        -0.466387        -1.961966        -0.768966              3          5.647100          9.573668         17.711298
1         2.907426         1.041547        -2.534225         1.033480        -1.286341         6.208372        -1.656764         3.034321              1         27.537777         -2.683659        -65.898262
2         2.909212         1.744247        -1.583371         1.825540        -0.306463         2.587239         0.286811         2.006733              1         32.439518         -2.136946        -53.056122
3         2.968790         1.307310        -2.042675         0.444853        -0.356500         2.745625        -1.558250       

# Clustering Experiment

https://scikit-learn.org/stable/modules/clustering.html  
Geometry *appears* to be spherical, which would mean non-flat. Which would lend to:
1. Affinity Propagation
1. Mean-shift
1. Spectral Clustering
1. DBSCAN
1. HDBSCAN
1. OPTICS

Not sure on number of clusters (few vs. many). I don't see many obvious clusters based on PCA / t-SNE visualization

The last 3 are density based and suited to very large N samples, which isn't this

In [43]:


# https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html

X = pd.concat([pca_component_df, tsne_df], axis=1)
X.index = offense_tendencies.index
print(X.head().to_string())

## Parameters

params = {
    "quantile": 0.3,
    "eps": 0.3,
    "damping": 0.9,
    "preference": -200,
    "n_neighbors": 3,
    "n_clusters": 3,
    "min_samples": 7,
    "xi": 0.05,
    "min_cluster_size": 0.1,
    "allow_single_cluster": True,
    "hdbscan_min_cluster_size": 15,
    "hdbscan_min_samples": 3,
    "random_state": 42,
}

# estimate bandwidth for mean shift
bandwidth = estimate_bandwidth(X, quantile=params["quantile"])

## Models ##

affinity_propagation = AffinityPropagation(
    damping=params["damping"],
    preference=params["preference"],
    random_state=params["random_state"],
)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)

spectral = SpectralClustering(
    n_clusters=params["n_clusters"],
    eigen_solver="arpack",
    affinity="nearest_neighbors",
    random_state=params["random_state"],
)
dbscan = DBSCAN(eps=params["eps"])

hdbscan = HDBSCAN(
    min_samples=params["hdbscan_min_samples"],
    min_cluster_size=params["hdbscan_min_cluster_size"],
    allow_single_cluster=params["allow_single_cluster"],
    copy=True,
)

optics = OPTICS(
    min_samples=params["min_samples"],
    xi=params["xi"],
    min_cluster_size=params["min_cluster_size"],
)

clustering_algorithms = (
    ("Affinity\nPropagation", affinity_propagation),
    ("MeanShift", ms),
    ("Spectral\nClustering", spectral),
    ("DBSCAN", dbscan),
    ("HDBSCAN", hdbscan),
    ("OPTICS", optics)
)

## Evaluate ##

for name, algorithm in clustering_algorithms:
    t0 = time()

    # catch warnings related to kneighbors_graph
    # with warnings.catch_warnings():
    #     warnings.filterwarnings(
    #         "ignore",
    #         message="the number of connected components of the "
    #         "connectivity matrix is [0-9]{1,2}"
    #         " > 1. Completing it to avoid stopping the tree early.",
    #         category=UserWarning,
    #     )
    #     warnings.filterwarnings(
    #         "ignore",
    #         message="Graph is not fully connected, spectral embedding"
    #         " may not work as expected.",
    #         category=UserWarning,
    #     )
    algorithm.fit(X)

    t1 = time()
    y_pred = None
    if hasattr(algorithm, "labels_"):
        y_pred = algorithm.labels_.astype(int)
    else:
        y_pred = algorithm.predict(X)

    offense_tendencies[f'{name} Cluster'] = y_pred

    # Visualize
    fig = px.scatter(
        x=X['TSNE Component 1'],
        y=X['TSNE Component 2'],
        color=y_pred,
        title=name
    )
    # fig = px.scatter_3d(
    #     x=X[X.columns[0]],
    #     y=X[X.columns[1]],
    #     z=X[X.columns[2]],
    #     color=y_pred,
    #     title=name
    # )
    fig.update_layout(
        xaxis_title='TSNE Component 1',
        yaxis_title='TSNE Component 2',
        # zaxis_title=X.columns[2],
    )
    fig.show()

                PCA Component 1  PCA Component 2  PCA Component 3  PCA Component 4  PCA Component 5  PCA Component 6  PCA Component 7  PCA Component 8  TSNE Component 1  TSNE Component 2
posteam season                                                                                                                                                                            
ARI     2018          -1.174916        -1.414879         1.749422        -0.534725         0.874502        -0.466387        -1.961966        -0.768966          6.305297         -1.726348
        2019           2.907426         1.041547        -2.534225         1.033480        -1.286341         6.208372        -1.656764         3.034321        -11.902147         -1.672906
        2020           2.909212         1.744247        -1.583371         1.825540        -0.306463         2.587239         0.286811         2.006733        -11.353816         -1.134172
        2021           2.968790         1.307310        -2.042675